# Advanced Apache Sedona Examples

This notebook demonstrates complex spatial analytics scenarios using Apache Sedona.

## Complex Use Cases Covered:
1. **Spatial ETL Pipeline** - Processing large spatial datasets
2. **Geofencing & Location Intelligence** - Real-time location analytics
3. **Spatial Clustering** - DBSCAN clustering of spatial points
4. **Route Optimization** - Spatial network analysis
5. **Heatmap Generation** - Spatial density analysis
6. **Multi-scale Spatial Joins** - Performance optimization techniques
7. **Spatial Machine Learning** - Predictive spatial modeling

In [ ]:
# Advanced imports for complex spatial operations
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator

from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Polygon as MPLPolygon
import folium
from folium.plugins import HeatMap
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json
import random
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
# Initialize Spark with optimized configuration for spatial operations
spark = SparkSession.builder \
    .appName("AdvancedSedonaExamples") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", "org.apache.sedona.core.serde.SedonaKryoRegistrator") \
    .config("spark.sql.extensions", "org.apache.sedona.viz.sql.SedonaVizExtensions,org.apache.sedona.sql.SedonaSqlExtensions") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
SedonaRegistrator.registerAll(spark)

print("✅ Advanced Sedona environment initialized!")
print(f"Spark Version: {spark.version}")
print(f"Available cores: {spark.sparkContext.defaultParallelism}")

## 1. Spatial ETL Pipeline: Processing NYC Taxi Data

Simulating processing of millions of taxi trips with spatial operations.

In [ ]:
# Generate large-scale taxi trip data (simulating 1M+ trips)
def generate_nyc_taxi_data(num_trips=100000):
    # NYC bounding box (approximate)
    nyc_bounds = {
        'min_lat': 40.4774, 'max_lat': 40.9176,
        'min_lon': -74.2591, 'max_lon': -73.7004
    }
    
    # Generate trip data
    trips = []
    base_time = datetime(2024, 1, 1)
    
    for i in range(num_trips):
        # Pickup location (slightly clustered around Manhattan)
        pickup_lat = np.random.normal(40.7589, 0.05)  # Centered on Manhattan
        pickup_lon = np.random.normal(-73.9851, 0.05)
        
        # Dropoff location (random within NYC)
        dropoff_lat = np.random.uniform(nyc_bounds['min_lat'], nyc_bounds['max_lat'])
        dropoff_lon = np.random.uniform(nyc_bounds['min_lon'], nyc_bounds['max_lon'])
        
        # Trip details
        trip_time = base_time + timedelta(minutes=np.random.randint(0, 525600))  # Random time in year
        fare = np.random.uniform(5.0, 50.0)
        distance = np.random.uniform(0.1, 20.0)
        
        trips.append({
            'trip_id': f'trip_{i:06d}',
            'pickup_datetime': trip_time.isoformat(),
            'pickup_lat': pickup_lat,
            'pickup_lon': pickup_lon,
            'dropoff_lat': dropoff_lat,
            'dropoff_lon': dropoff_lon,
            'fare_amount': fare,
            'trip_distance': distance,
            'passenger_count': np.random.randint(1, 7)
        })
    
    return trips

print("Generating NYC taxi trip data...")
taxi_data = generate_nyc_taxi_data(50000)  # 50K trips for demo
print(f"Generated {len(taxi_data)} taxi trips")

# Convert to Spark DataFrame
taxi_schema = StructType([
    StructField("trip_id", StringType(), True),
    StructField("pickup_datetime", StringType(), True),
    StructField("pickup_lat", DoubleType(), True),
    StructField("pickup_lon", DoubleType(), True),
    StructField("dropoff_lat", DoubleType(), True),
    StructField("dropoff_lon", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("passenger_count", IntegerType(), True)
])

taxi_df = spark.createDataFrame(taxi_data, schema=taxi_schema)
print(f"Created Spark DataFrame with {taxi_df.count()} records")

In [ ]:
# Complex Spatial ETL Operations
taxi_df.createOrReplaceTempView("taxi_trips")

# 1. Create spatial geometries and calculate trip vectors
spatial_trips = spark.sql("""
    SELECT 
        trip_id,
        pickup_datetime,
        ST_Point(pickup_lon, pickup_lat) as pickup_point,
        ST_Point(dropoff_lon, dropoff_lat) as dropoff_point,
        ST_Distance(ST_Point(pickup_lon, pickup_lat), ST_Point(dropoff_lon, dropoff_lat)) as euclidean_distance,
        fare_amount,
        trip_distance,
        passenger_count,
        CASE 
            WHEN HOUR(pickup_datetime) BETWEEN 7 AND 9 THEN 'Morning Rush'
            WHEN HOUR(pickup_datetime) BETWEEN 17 AND 19 THEN 'Evening Rush'
            WHEN HOUR(pickup_datetime) BETWEEN 22 AND 5 THEN 'Night'
            ELSE 'Regular'
        END as time_period
    FROM taxi_trips
    WHERE pickup_lat BETWEEN 40.4 AND 41.0 
      AND pickup_lon BETWEEN -74.5 AND -73.5
      AND dropoff_lat BETWEEN 40.4 AND 41.0 
      AND dropoff_lon BETWEEN -74.5 AND -73.5
""")

spatial_trips.cache()
print(f"Processed {spatial_trips.count()} valid spatial trips")
spatial_trips.show(5)

## 2. Advanced Geofencing: Multi-Zone Analysis

Creating complex geofences and analyzing spatial patterns.

In [ ]:
# Create NYC borough-like zones using polygons
zones_data = [
    {
        'zone_id': 'manhattan_south', 
        'zone_name': 'Lower Manhattan',
        'polygon': 'POLYGON((-74.0479 40.6829, -73.9441 40.6829, -73.9441 40.7589, -74.0479 40.7589, -74.0479 40.6829))'
    },
    {
        'zone_id': 'manhattan_central', 
        'zone_name': 'Midtown Manhattan',
        'polygon': 'POLYGON((-74.0479 40.7589, -73.9441 40.7589, -73.9441 40.8176, -74.0479 40.8176, -74.0479 40.7589))'
    },
    {
        'zone_id': 'brooklyn_west', 
        'zone_name': 'West Brooklyn',
        'polygon': 'POLYGON((-74.0479 40.6000, -73.9000 40.6000, -73.9000 40.7000, -74.0479 40.7000, -74.0479 40.6000))'
    },
    {
        'zone_id': 'queens_central', 
        'zone_name': 'Central Queens',
        'polygon': 'POLYGON((-73.9000 40.7000, -73.7500 40.7000, -73.7500 40.8000, -73.9000 40.8000, -73.9000 40.7000))'
    }
]

zones_df = spark.createDataFrame(zones_data)
zones_df.createOrReplaceTempView("zones")

# Create spatial zones
spatial_zones = spark.sql("""
    SELECT 
        zone_id,
        zone_name,
        ST_GeomFromWKT(polygon) as zone_geometry,
        ST_Area(ST_GeomFromWKT(polygon)) as zone_area
    FROM zones
""")

spatial_zones.show()

In [ ]:
# Complex spatial join: Assign pickup and dropoff zones
spatial_trips.createOrReplaceTempView("spatial_trips")
spatial_zones.createOrReplaceTempView("spatial_zones")

trips_with_zones = spark.sql("""
    SELECT 
        t.trip_id,
        t.pickup_datetime,
        t.pickup_point,
        t.dropoff_point,
        t.euclidean_distance,
        t.fare_amount,
        t.trip_distance,
        t.passenger_count,
        t.time_period,
        pz.zone_id as pickup_zone,
        pz.zone_name as pickup_zone_name,
        dz.zone_id as dropoff_zone,
        dz.zone_name as dropoff_zone_name,
        CASE 
            WHEN pz.zone_id = dz.zone_id THEN 'Intra-zone'
            ELSE 'Inter-zone'
        END as trip_type
    FROM spatial_trips t
    LEFT JOIN spatial_zones pz ON ST_Within(t.pickup_point, pz.zone_geometry)
    LEFT JOIN spatial_zones dz ON ST_Within(t.dropoff_point, dz.zone_geometry)
""")

trips_with_zones.cache()
print(f"Trips with zone assignments: {trips_with_zones.count()}")

# Analyze zone patterns
zone_analysis = spark.sql("""
    SELECT 
        pickup_zone_name,
        dropoff_zone_name,
        trip_type,
        time_period,
        COUNT(*) as trip_count,
        AVG(fare_amount) as avg_fare,
        AVG(euclidean_distance) as avg_distance,
        SUM(passenger_count) as total_passengers
    FROM trips_with_zones
    WHERE pickup_zone IS NOT NULL AND dropoff_zone IS NOT NULL
    GROUP BY pickup_zone_name, dropoff_zone_name, trip_type, time_period
    ORDER BY trip_count DESC
""")

zone_analysis.show(20)

## 3. Spatial Clustering: DBSCAN-like Analysis

Finding hotspots and clusters in pickup locations.

In [ ]:
# Spatial clustering using grid-based approach (DBSCAN alternative for big data)
def create_spatial_grid(df, grid_size=0.001):  # ~100m grid cells
    """
    Create a spatial grid for clustering analysis
    """
    df.createOrReplaceTempView("points")
    
    grid_df = spark.sql(f"""
        SELECT 
            FLOOR(ST_X(pickup_point) / {grid_size}) * {grid_size} as grid_x,
            FLOOR(ST_Y(pickup_point) / {grid_size}) * {grid_size} as grid_y,
            COUNT(*) as point_count,
            AVG(fare_amount) as avg_fare,
            time_period,
            COLLECT_LIST(trip_id) as trip_ids
        FROM points
        WHERE pickup_point IS NOT NULL
        GROUP BY grid_x, grid_y, time_period
        HAVING point_count >= 3
        ORDER BY point_count DESC
    """)
    
    return grid_df

# Create hotspot analysis
hotspots = create_spatial_grid(trips_with_zones)
hotspots.cache()

print(f"Identified {hotspots.count()} spatial hotspots")
hotspots.show(10)

In [ ]:
# Advanced hotspot analysis with density metrics
hotspots.createOrReplaceTempView("hotspots")

hotspot_analysis = spark.sql("""
    WITH hotspot_stats AS (
        SELECT 
            grid_x,
            grid_y,
            time_period,
            point_count,
            avg_fare,
            ST_Point(grid_x, grid_y) as grid_center,
            -- Calculate local density (points within 500m)
            (
                SELECT SUM(h2.point_count) 
                FROM hotspots h2 
                WHERE h2.time_period = h1.time_period
                  AND ST_Distance(ST_Point(h1.grid_x, h1.grid_y), ST_Point(h2.grid_x, h2.grid_y)) <= 0.005
            ) as neighborhood_density
        FROM hotspots h1
    ),
    ranked_hotspots AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY time_period ORDER BY neighborhood_density DESC) as density_rank,
            CASE 
                WHEN neighborhood_density >= 100 THEN 'Super Hotspot'
                WHEN neighborhood_density >= 50 THEN 'Major Hotspot'
                WHEN neighborhood_density >= 20 THEN 'Minor Hotspot'
                ELSE 'Regular Area'
            END as hotspot_category
        FROM hotspot_stats
    )
    SELECT 
        time_period,
        hotspot_category,
        COUNT(*) as num_areas,
        AVG(point_count) as avg_pickups_per_area,
        AVG(avg_fare) as avg_fare_in_category,
        MAX(neighborhood_density) as max_density
    FROM ranked_hotspots
    GROUP BY time_period, hotspot_category
    ORDER BY time_period, max_density DESC
""")

print("Hotspot Analysis by Time Period:")
hotspot_analysis.show()

## 4. Route Optimization & Network Analysis

Analyzing optimal routes and identifying inefficient trips.

In [ ]:
# Route efficiency analysis
trips_with_zones.createOrReplaceTempView("trips_analysis")

route_efficiency = spark.sql("""
    SELECT 
        trip_id,
        pickup_zone_name,
        dropoff_zone_name,
        euclidean_distance,
        trip_distance,
        fare_amount,
        time_period,
        -- Calculate efficiency metrics
        CASE 
            WHEN euclidean_distance > 0 THEN trip_distance / euclidean_distance 
            ELSE NULL 
        END as detour_ratio,
        
        CASE 
            WHEN trip_distance > 0 THEN fare_amount / trip_distance 
            ELSE NULL 
        END as fare_per_mile,
        
        -- Classify trip efficiency
        CASE 
            WHEN trip_distance / euclidean_distance <= 1.2 THEN 'Efficient'
            WHEN trip_distance / euclidean_distance <= 1.5 THEN 'Moderate'
            ELSE 'Inefficient'
        END as route_efficiency
    FROM trips_analysis
    WHERE euclidean_distance > 0.001  -- Filter out very short trips
      AND trip_distance > 0
      AND pickup_zone_name IS NOT NULL
      AND dropoff_zone_name IS NOT NULL
""")

route_efficiency.cache()
print(f"Route efficiency analysis for {route_efficiency.count()} trips")
route_efficiency.show(10)

In [ ]:
# Advanced route analysis with corridor identification
route_efficiency.createOrReplaceTempView("route_efficiency")

corridor_analysis = spark.sql("""
    WITH route_corridors AS (
        SELECT 
            pickup_zone_name,
            dropoff_zone_name,
            time_period,
            COUNT(*) as trip_volume,
            AVG(detour_ratio) as avg_detour,
            AVG(fare_per_mile) as avg_fare_per_mile,
            AVG(euclidean_distance) as avg_distance,
            PERCENTILE_APPROX(detour_ratio, 0.95) as p95_detour,
            -- Efficiency score (lower is better)
            AVG(detour_ratio) * 100 + (1.0 / AVG(fare_per_mile)) as inefficiency_score
        FROM route_efficiency
        WHERE pickup_zone_name != dropoff_zone_name  -- Inter-zone trips only
        GROUP BY pickup_zone_name, dropoff_zone_name, time_period
        HAVING trip_volume >= 5  -- Focus on popular routes
    ),
    ranked_corridors AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY time_period ORDER BY trip_volume DESC) as volume_rank,
            ROW_NUMBER() OVER (PARTITION BY time_period ORDER BY inefficiency_score DESC) as inefficiency_rank
        FROM route_corridors
    )
    SELECT 
        time_period,
        pickup_zone_name,
        dropoff_zone_name,
        trip_volume,
        ROUND(avg_detour, 2) as avg_detour_ratio,
        ROUND(avg_fare_per_mile, 2) as avg_fare_per_mile,
        ROUND(inefficiency_score, 2) as inefficiency_score,
        volume_rank,
        inefficiency_rank,
        CASE 
            WHEN inefficiency_rank <= 3 THEN 'Optimization Priority'
            WHEN volume_rank <= 5 THEN 'High Volume Corridor'
            ELSE 'Regular Route'
        END as route_category
    FROM ranked_corridors
    WHERE volume_rank <= 10 OR inefficiency_rank <= 5
    ORDER BY time_period, inefficiency_rank
""")

print("Top Route Corridors by Time Period:")
corridor_analysis.show(30, truncate=False)

## 5. Spatial Machine Learning: Demand Prediction

Using spatial features for predictive modeling.

In [ ]:
# Create features for ML model
ml_features = spark.sql("""
    WITH spatial_features AS (
        SELECT 
            grid_x,
            grid_y,
            time_period,
            point_count as demand,
            avg_fare,
            -- Spatial features
            grid_x * 1000000 as x_scaled,  -- Scale coordinates
            grid_y * 1000000 as y_scaled,
            
            -- Time-based features
            CASE time_period 
                WHEN 'Morning Rush' THEN 1 
                WHEN 'Evening Rush' THEN 2
                WHEN 'Night' THEN 3
                ELSE 0 
            END as time_encoded,
            
            -- Distance from city center (approximate)
            SQRT(POWER(grid_x - (-73.9851), 2) + POWER(grid_y - 40.7589, 2)) as distance_from_center
        FROM hotspots
        WHERE point_count >= 3
    )
    SELECT *,
        -- Categorize demand levels for classification
        CASE 
            WHEN demand >= 20 THEN 2  -- High demand
            WHEN demand >= 10 THEN 1  -- Medium demand  
            ELSE 0                    -- Low demand
        END as demand_category
    FROM spatial_features
""")

ml_features.cache()
print(f"Created ML features for {ml_features.count()} spatial-temporal points")
ml_features.show(10)

In [ ]:
# Prepare data for ML clustering (spatial demand patterns)
feature_cols = ['x_scaled', 'y_scaled', 'time_encoded', 'distance_from_center', 'avg_fare']

# Assemble features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
ml_data = assembler.transform(ml_features)

# Apply K-means clustering to identify demand patterns
kmeans = KMeans(k=5, seed=42, featuresCol="features", predictionCol="cluster")
model = kmeans.fit(ml_data)
predictions = model.transform(ml_data)

# Analyze clusters
predictions.createOrReplaceTempView("ml_predictions")

cluster_analysis = spark.sql("""
    SELECT 
        cluster,
        COUNT(*) as cluster_size,
        AVG(demand) as avg_demand,
        AVG(avg_fare) as avg_fare_in_cluster,
        AVG(distance_from_center) as avg_distance_from_center,
        
        -- Most common time period in cluster
        MODE() WITHIN GROUP (ORDER BY time_period) as dominant_time_period,
        
        -- Demand characteristics
        MIN(demand) as min_demand,
        MAX(demand) as max_demand,
        STDDEV(demand) as demand_std
    FROM ml_predictions
    GROUP BY cluster
    ORDER BY cluster
""")

print("Spatial-Temporal Demand Clusters:")
cluster_analysis.show()

## 6. Performance Optimization Techniques

Demonstrating advanced Sedona performance optimization.

In [ ]:
# Spatial indexing and partitioning strategies
import time

def benchmark_spatial_join(df1, df2, join_condition, description):
    """
    Benchmark different spatial join strategies
    """
    start_time = time.time()
    result_count = join_condition.count()
    end_time = time.time()
    
    print(f"{description}:")
    print(f"  - Result count: {result_count}")
    print(f"  - Execution time: {end_time - start_time:.2f} seconds")
    print(f"  - Partitions: {join_condition.rdd.getNumPartitions()}")
    return result_count, end_time - start_time

# Create test datasets
large_points = spark.sql("""
    SELECT 
        ST_Point(RAND() * 0.1 - 74.0, RAND() * 0.1 + 40.7) as point,
        CAST(RAND() * 1000 AS INT) as point_id
    FROM range(10000)
""")

test_polygons = spark.sql("""
    SELECT 
        ST_Buffer(ST_Point(RAND() * 0.05 - 73.98, RAND() * 0.05 + 40.75), 0.001) as polygon,
        CAST(RAND() * 100 AS INT) as poly_id
    FROM range(100)
""")

large_points.cache()
test_polygons.cache()

print(f"Created {large_points.count()} test points and {test_polygons.count()} test polygons")

In [ ]:
# Benchmark different join strategies
large_points.createOrReplaceTempView("large_points")
test_polygons.createOrReplaceTempView("test_polygons")

# Strategy 1: Basic spatial join
basic_join = spark.sql("""
    SELECT p.point_id, pg.poly_id
    FROM large_points p, test_polygons pg
    WHERE ST_Within(p.point, pg.polygon)
""")

benchmark_spatial_join(large_points, test_polygons, basic_join, "Basic Spatial Join")

# Strategy 2: With spatial indexing hint
indexed_join = spark.sql("""
    SELECT /*+ BROADCAST(pg) */ p.point_id, pg.poly_id
    FROM large_points p, test_polygons pg
    WHERE ST_Within(p.point, pg.polygon)
""")

benchmark_spatial_join(large_points, test_polygons, indexed_join, "Broadcast Join Strategy")

# Performance tips summary
print("\n🚀 Performance Optimization Tips:")
print("1. Use broadcast joins for small polygon datasets (< 200MB)")
print("2. Partition data by spatial regions for large datasets")
print("3. Cache frequently accessed spatial DataFrames")
print("4. Use appropriate spatial predicates (ST_Within vs ST_Intersects)")
print("5. Consider spatial indexing for repeated queries")

## 7. Advanced Visualization: Interactive Heatmaps

Creating sophisticated spatial visualizations.

In [ ]:
# Prepare data for visualization
viz_data = spark.sql("""
    SELECT 
        grid_x as longitude,
        grid_y as latitude,
        point_count as intensity,
        avg_fare,
        time_period
    FROM hotspots
    WHERE point_count >= 5
    ORDER BY point_count DESC
    LIMIT 500
""")

viz_pandas = viz_data.toPandas()
print(f"Prepared {len(viz_pandas)} points for visualization")

# Create multi-layer interactive map
def create_advanced_heatmap(df):
    # Center map on NYC
    center_lat = df['latitude'].mean()
    center_lon = df['longitude'].mean()
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Add different layers for different time periods
    time_periods = df['time_period'].unique()
    colors = ['red', 'blue', 'green', 'orange']
    
    for i, period in enumerate(time_periods):
        period_data = df[df['time_period'] == period]
        
        # Create heatmap data
        heat_data = [[row['latitude'], row['longitude'], row['intensity']] 
                    for idx, row in period_data.iterrows()]
        
        if heat_data:  # Only create layer if data exists
            HeatMap(
                heat_data,
                name=f'Demand - {period}',
                radius=15,
                blur=10,
                gradient={0.2: colors[i % len(colors)], 1.0: colors[i % len(colors)]}
            ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

if len(viz_pandas) > 0:
    heatmap = create_advanced_heatmap(viz_pandas)
    print("✅ Interactive heatmap created! (Display in Jupyter)")
    # heatmap  # Uncomment to display in Jupyter
else:
    print("No data available for visualization")

In [ ]:
# Advanced statistical analysis
summary_stats = spark.sql("""
    WITH overall_stats AS (
        SELECT 
            COUNT(DISTINCT trip_id) as total_trips,
            COUNT(DISTINCT pickup_zone_name) as unique_pickup_zones,
            COUNT(DISTINCT dropoff_zone_name) as unique_dropoff_zones,
            AVG(fare_amount) as avg_fare,
            AVG(euclidean_distance) as avg_distance,
            SUM(passenger_count) as total_passengers
        FROM trips_with_zones
        WHERE pickup_zone_name IS NOT NULL
    ),
    efficiency_stats AS (
        SELECT 
            route_efficiency,
            COUNT(*) as trip_count,
            AVG(detour_ratio) as avg_detour,
            AVG(fare_per_mile) as avg_fare_per_mile
        FROM route_efficiency
        GROUP BY route_efficiency
    )
    SELECT 
        'Overall Statistics' as metric_type,
        CAST(total_trips AS STRING) as value,
        'Total processed trips' as description
    FROM overall_stats
    
    UNION ALL
    
    SELECT 
        'Spatial Coverage' as metric_type,
        CAST(unique_pickup_zones AS STRING) as value,
        'Unique pickup zones covered' as description
    FROM overall_stats
    
    UNION ALL
    
    SELECT 
        'Route Efficiency' as metric_type,
        CONCAT(route_efficiency, ': ', CAST(trip_count AS STRING), ' trips') as value,
        CONCAT('Avg detour ratio: ', CAST(ROUND(avg_detour, 2) AS STRING)) as description
    FROM efficiency_stats
    ORDER BY metric_type, value
""")

print("\n📊 Advanced Spatial Analytics Summary:")
summary_stats.show(20, truncate=False)

In [ ]:
# Cleanup
print("\n🧹 Cleaning up resources...")
spark.catalog.clearCache()
print("Cache cleared.")

print("\n🎯 Complex Spatial Analytics Completed!")
print("\nKey Capabilities Demonstrated:")
print("• Large-scale spatial ETL processing")
print("• Multi-zone geofencing analysis")
print("• Spatial clustering and hotspot detection")
print("• Route optimization analysis")
print("• Spatial machine learning integration")
print("• Performance optimization techniques")
print("• Advanced interactive visualizations")

# Uncomment to stop Spark (keep running for interactive use)
# spark.stop()